In [ ]:
%cd ..

In [ ]:
from time import sleep

import numpy as np
import matplotlib.pyplot as plt

from evgena.dataset import Dataset
from evgena.genetals import Images2LabelObjectiveFnc, DecayImageSampler, TrivialImageSampler, BinomialInit, ClipOperator, BinomialMutation, FGSMMutation
from evgena.model import TrainableTfModel
from evgena.metrics import mse
from evgena.utils.large_files import maybe_download
from evgena.data_transformations import images_to_BHWC, augment_images
from genetals.core import *
from genetals.initializers import MultivariateRandomInit
from genetals.operators import ShuffleOperator, TwoPointXover, BiasedMutation, NSGAOperator, UniversalXover
from genetals.callbacks import MultiObjectiveReport, GAStatus


class TooGoodEarlyStopping(EarlyStoppingBase):
    def __init__(self, prediction_bound: float = 0.5, similarity_bound: float = -0.001):
        super(TooGoodEarlyStopping, self).__init__()
        
        self._ticks = 0
        self._results = []
        self._prediction_bound = prediction_bound
        self._similarity_bound = similarity_bound

    def __call__(self, ga: 'GeneticAlgorithm') -> bool:
        curr_pop = ga.capture(-1)
        
        success = np.all(
            curr_pop.objectives > [self._prediction_bound, self._similarity_bound], axis=-1
        )
        if np.any(success):
            self._results.append((curr_pop.genes[success].copy(), curr_pop.objectives[success].copy(), ga.current_generation))
            return True
            
        return False
    
    def tick(self):
        self._ticks += 1
        while len(self._results) < self._ticks:
            self._results.append((None, None, None))

# Images to label #
Demonstration of GA ability to generate adversarial pattern for misclassifying a set of images to given label.
- [experiment](#Experiment)
- [results inspection](#Results-inspection)

## Experiment ##

Model to be "attacked" and set of images to convert

In [ ]:
# model_path = 'models/fashion_mnist_densenet/fold_0'
model_path = 'models/fashion_mnist_simplenet/fold_7'

model = TrainableTfModel(model_path)
fashion_mnist = Dataset.from_nprecord('datasets/stratified_fashion_mnist.npz')

In [ ]:
source_class = 0
target_class = 1
source_dataset = Dataset.sub_dataset(fashion_mnist, [source_class], do_stratified=False)

target_images = source_dataset.test.X
image_sampler = TrivialImageSampler(target_images[:1])  # singleton attack
# image_sampler = DecayImageSampler(target_images, sample_size=32, decay=0.99)  # universal attack
dataset = image_sampler._source_images

### GA definition ###

In [ ]:
%matplotlib notebook

# visualization
fig, ax = plt.subplots(1, 1, figsize=(8,4))

ax.set_xlim(0.0000000001, 1)
ax.set_xlabel('Target class prediction probability')
ax.set_ylim(-0.01, 0)
ax.set_ylabel('similarity')
ax.set_xscale('log')
ax.grid(axis='both')
ax.vlines(0.5, -1, 1, colors='g')

initializer = BinomialInit((28, 28), (1/255), [128, 1024, 4096])

# GA graph definition
graph = OperatorGraph()
select_op = ShuffleOperator(graph.init_op)
xover_op = TwoPointXover(select_op, 0.8)
mutation_op = BinomialMutation(
    xover_op, individual_mut_ratio=0.3, gene_mut_ratio=0.15,
    step_size=(1/255), max_steps=26, mean_steps=0
)
clip_op = ClipOperator(mutation_op, -0.2, 0.2)
moea_op = NSGAOperator(graph.init_op, clip_op)

callbacks = [MultiObjectiveReport(ax), GAStatus(fig), lambda *_: image_sampler.next()]

# GA initialization
ga = GeneticAlgorithm(
    initializer=initializer,
    operator_graph=graph,
    objective_fnc=Images2LabelObjectiveFnc(
        model, target_class, image_sampler, lambda x, y: - np.maximum(mse(x, y), 0.0005)
    ),
    # early_stopping=TooGoodEarlyStopping(similarity_bound=-0.005),  # comment if you early stopping undesirable
    callbacks=callbacks,
    metadata={
        'dataset': target_images,
        'model': np.asarray(model_path),
        'source_class': np.asarray(source_class),
        'target_class': np.asarray(target_class)
    },
    results_dir='logs/images2label'
)

### GA run ###

In [ ]:
%time final_pop, fitnesses, objectives = ga.run(population_size=256, generation_cap=256)

## Results inspection ##

In [ ]:
prediction_bound = 0.5

augmented_images = augment_images(final_pop.genes, dataset)
augmented_images_batch_shaped = augmented_images.reshape(-1, *augmented_images.shape[2:4], 1)

generalization = model(augmented_images_batch_shaped)[:, target_class].reshape(augmented_images.shape[:2])

print('Mean test prediction: {:4f}, target predicted {} out of {}.'.format(generalization.mean(), np.sum(generalization > 0.5), generalization.size))

filtered_indices, *_ = np.where(
    generalization.max(axis=-1) > prediction_bound
)

print('Successfull test prediction: {:4f}, target predicted {} out of {}'.format(generalization[filtered_indices].mean(), np.sum(generalization[filtered_indices] > 0.5), generalization[filtered_indices].size))

filtered_individuals = final_pop.genes[filtered_indices]
leaderboard = filtered_indices[np.flip(np.argsort(generalization[filtered_indices].sum(axis=-1)), 0)]

%matplotlib ipympl

compare_fig, compare_ax = plt.subplots(1, 3, figsize=(8, 3))
compare_fig.tight_layout()

In [ ]:
# individual_rank = -1  # smallest feasible perturbation
individual_rank = 0  # smallest feasible error

individual_i = leaderboard[individual_rank]
image_i = np.where(generalization[leaderboard[individual_rank]] > 0.5)[0][0]

compare_ax[0].imshow(dataset[image_i], cmap='gray', vmin=0, vmax=1)
compare_ax[1].imshow(final_pop.genes[individual_i], cmap='plasma', vmin=-1, vmax=1)
compare_ax[2].imshow(augmented_images[individual_i, image_i][:,:,0], cmap='gray', vmin=0, vmax=1)

compare_fig.canvas.set_window_title('Target score: {}'.format(
    model(augmented_images[individual_i, image_i])[0, target_class]
))
compare_fig.canvas.draw()